### **Visão Geral e Objetivo do Notebook**

Este notebook demonstra uma implementação de **Blind Quantum Computing (BQC)**, um paradigma que permite a um cliente (Alice) executar um algoritmo quântico em um servidor quântico remoto (Bob) sem revelar informações sobre o algoritmo ou os dados.

Para isso, o código utiliza um **Classificador Quântico Variacional (VQC)**, um modelo de Machine Learning híbrido quântico-clássico. O objetivo é classificar o famoso dataset Iris (com 3 classes) enquanto simula o processo de envio do circuito quântico a uma rede a cada época de treinamento, mimetizando um cenário de BQC.

As principais tecnologias utilizadas são:
* **Qiskit:** Para a construção dos circuitos quânticos e a definição do modelo de Machine Learning.
* **Qiskit Aer:** Para simular a execução do circuito em um backend com ruído, tornando o ambiente mais realista.
* **Scikit-learn:** Para o pré-processamento dos dados e o cálculo das métricas de avaliação.
* **Quantumnet:** Uma biblioteca customizada para simular a topologia de rede e a transmissão do circuito, abstraindo a complexidade do BQC.

### **Parte 1: Importações e Configuração Inicial**

Nesta primeira célula de código, realizamos duas tarefas essenciais:

1.  **Instalação de Dependências:** O comando `!pip install` garante que todas as bibliotecas necessárias (qiskit, qiskit-aer, qiskit-machine-learning, qiskit-algorithms, scikit-learn e numpy) estejam instaladas no ambiente de execução.

2.  **Importação de Módulos:** Em seguida, importamos as classes e funções específicas que serão usadas ao longo do código. Isso inclui ferramentas para construir circuitos (`QuantumCircuit`, `ZFeatureMap`) e modelos de Machine Learning (`EstimatorQNN`, `NeuralNetworkClassifier`), bem como utilitários para processamento de dados (`load_iris`, `MinMaxScaler`) e a biblioteca `quantumnet` para a simulação de BQC.

Finalmente, uma semente de aleatoriedade (`SEED = 42`) é definida para `random` e `numpy`. Isso é crucial para garantir que os resultados sejam reproduzíveis; ou seja, sempre que o código for executado, a divisão dos dados e a inicialização dos pesos aleatórios serão as mesmas.

In [ ]:
# Passo 1: Importações e Configuração
import os
import random
import numpy as np
import matplotlib.pyplot as plt
import time

# Importações do Qiskit
from qiskit import QuantumCircuit
from qiskit.circuit.library import ZFeatureMap, RealAmplitudes
from qiskit_aer.primitives import Estimator as AerEstimator
from qiskit_aer.noise import NoiseModel, depolarizing_error
from qiskit.quantum_info import SparsePauliOp
from qiskit_machine_learning.neural_networks import EstimatorQNN
from qiskit_machine_learning.algorithms.classifiers import NeuralNetworkClassifier
from qiskit_machine_learning.utils.loss_functions import CrossEntropyLoss
from qiskit_algorithms.optimizers import SPSA

# Importações do Scikit-learn
from sklearn.datasets import load_iris
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, ConfusionMatrixDisplay

# Importação da biblioteca BQC (assumindo que está em um módulo chamado 'quantumnet')
# from quantumnet.components import Network, Logger

# Semente para reprodutibilidade
SEED = 42
random.seed(SEED)
np.random.seed(SEED)

### **Parte 2: Preparação do Dataset Iris**

O sucesso de qualquer modelo de Machine Learning começa com um bom tratamento dos dados. Nesta seção, o dataset Iris é preparado para ser utilizado pelo classificador quântico.

1.  **Carregamento:** `load_iris()` carrega o conjunto de dados, que contém informações sobre três espécies de flores Iris (setosa, versicolor e virginica) com base em quatro características (comprimento e largura da sépala e da pétala).

2.  **Normalização (Scaling):** As características (`X`) são redimensionadas para um intervalo entre 0 e 1 usando o `MinMaxScaler`. Este passo é fundamental para algoritmos quânticos, pois os parâmetros dos circuitos (como ângulos de rotação) operam em uma escala bem definida.

3.  **Codificação One-Hot:** O VQC que será construído produzirá uma saída para cada classe. Para comparar a previsão com o rótulo real, os rótulos de saída (`y`) são convertidos de um único número (0, 1 ou 2) para um formato vetorial, onde apenas a classe correta é marcada com '1' (ex: a classe 2 se torna `[0, 0, 1]`). Isso é feito pelo `OneHotEncoder`.

4.  **Divisão em Treino e Teste:** Os dados são divididos em um conjunto de treinamento (usado para ensinar o modelo) e um conjunto de teste (usado para avaliar seu desempenho em dados nunca vistos), utilizando a função `train_test_split`.

In [ ]:
# Passo 2: Preparação dos Dados (3 Classes)
iris = load_iris()
X = iris.data
y = iris.target.reshape(-1, 1)

# Codificar rótulos com One-Hot
encoder = OneHotEncoder(sparse_output=False)
y_onehot = encoder.fit_transform(y)

# Normalizar as características
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# Dividir dados em conjuntos de treino e teste
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y_onehot, test_size=0.2, random_state=SEED, stratify=y
)
print(f"Dados preparados para classificação multiclasse ({y_onehot.shape[1]} classes) com {X_train.shape[1]} features.")

### **Parte 3: Configuração do Backend com Ruído**

Computadores quânticos reais são suscetíveis a ruídos do ambiente, que podem causar erros nos cálculos. Para tornar a simulação mais próxima da realidade, configuramos um backend quântico simulado que imita esse comportamento.

1.  **Criação do Modelo de Ruído:** Um `NoiseModel` é instanciado. Nele, adicionamos um `depolarizing_error` (erro de despolarização), que é um tipo comum de ruído quântico. Definimos uma taxa de erro de 1% para portas de um único qubit (`error_1q`) e 2% para portas de dois qubits (`error_2q`).

2.  **Aplicação do Ruído:** O modelo de ruído é configurado para aplicar esses erros a portas quânticas específicas (`'rz'`, `'sx'`, `'x'` e `'cx'`).

3.  **Criação do Estimador Ruidoso:** O `AerEstimator` é o simulador do Qiskit que executará os circuitos. Nós o configuramos para usar o `noise_model` criado, garantindo que todas as execuções daqui para frente incluam os efeitos do ruído. As `run_options` definem que cada "tiro" (shot) do circuito será repetido 1024 vezes para obter uma estatística robusta do resultado.

In [ ]:
# Passo 3: Configuração do Backend com Ruído
print("\nConfigurando o simulador com modelo de ruído...")
noise_model = NoiseModel()
error_1q = depolarizing_error(0.01, 1)
error_2q = depolarizing_error(0.02, 2)

# Adicionar erros a portas específicas
noise_model.add_all_qubit_quantum_error(error_1q, ['rz', 'sx', 'x'])
noise_model.add_all_qubit_quantum_error(error_2q, ['cx'])

# Criar o estimador ruidoso
noisy_estimator = AerEstimator(
    backend_options={"noise_model": noise_model},
    run_options={"seed": SEED, "shots": 1024},
    transpile_options={"seed_transpiler": SEED},
)

### **Parte 4: Construção do Modelo Quântico (VQC)**

Aqui, montamos a arquitetura do nosso modelo de rede neural quântica. Ele é composto por um circuito quântico parametrizado e observáveis para extrair os resultados.

1.  **Feature Map (Mapa de Características):** O `ZFeatureMap` é a primeira parte do circuito. Sua função é codificar os dados clássicos de entrada (as 4 características das flores Iris) nos estados dos qubits.

2.  **Ansatz:** O `RealAmplitudes` é a segunda parte do circuito. Este é o componente "treinável" do modelo. Ele consiste em uma sequência de portas quânticas cujos parâmetros (ângulos de rotação) serão ajustados durante o treinamento pelo otimizador clássico para minimizar a função de perda. O argumento `reps=3` define a profundidade (ou complexidade) deste circuito.

3.  **Observáveis:** Para um problema de classificação com 3 classes, precisamos de 3 saídas. Os `observables` (`SparsePauliOp`) definem como os qubits serão medidos para produzir essas saídas. Aqui, medimos o operador Pauli Z em cada um dos três primeiros qubits, o que nos dará um valor esperado que será usado como a "ativação" da rede para cada classe.

4.  **EstimatorQNN:** Finalmente, o `EstimatorQNN` une todas as peças: o circuito (`qc`), os observáveis, a definição de quais parâmetros são da entrada (`input_params`) e quais são os pesos treináveis (`weight_params`), e o simulador com ruído (`noisy_estimator`). Ele representa a nossa rede neural quântica completa.

In [ ]:
# Passo 4: Construção do Modelo Quântico
num_qubits = X_train.shape[1]
feature_map = ZFeatureMap(num_qubits)
ansatz = RealAmplitudes(num_qubits, reps=3)

# Combinar feature map e ansatz em um único circuito
qc = QuantumCircuit(num_qubits)
qc.compose(feature_map, inplace=True)
qc.compose(ansatz, inplace=True)

# Definir observáveis para as 3 classes
observables = [
    SparsePauliOp("ZIII"),
    SparsePauliOp("IZII"),
    SparsePauliOp("IIZI")
]

# Criar a Rede Neural Quântica
qnn = EstimatorQNN(
    circuit=qc,
    observables=observables,
    input_params=feature_map.parameters,
    weight_params=ansatz.parameters,
    estimator=noisy_estimator
)

### **Parte 5: Simulação de Rede BQC e Função de Callback**

Esta é a seção central que simula o paradigma de **Blind Quantum Computing (BQC)**. A ideia é que, a cada passo do treinamento, o circuito com os pesos atualizados é "enviado" para uma rede remota para execução.

1.  **Inicialização da Rede (Conceitual):** O código original usa uma classe `Network` customizada. Para este notebook de propósito geral, definiremos uma função de espaço reservado. Em um cenário real de BQC, este passo inicializaria a conexão com o servidor quântico.

2.  **Função de Callback `simulacao_callback_bqc`:**
    * Um "callback" é uma função que é passada como argumento para outra e é executada automaticamente em um determinado momento.
    * Esta função foi projetada para ser chamada **ao final de cada época** do processo de treinamento do VQC.
    * Dentro dela, simulamos o processo de envio do circuito. Ela imprime a perda (loss) atual e o número da época, mimetizando a comunicação que ocorreria em um protocolo BQC real. Ela não executa o circuito para obter resultados (isso é trabalho do `EstimatorQNN`), mas sim simula a transmissão segura e privada do experimento a cada iteração.

In [ ]:
# Passo 5: Simulação de Rede BQC e Definição do Callback
print("\nDefinindo o callback para a simulação de BQC...")

# Este é um espaço reservado para a inicialização da rede do notebook original.
# Em um cenário real, isso configuraria as conexões de rede.
# rede = Network()
# rede.set_ready_topology('grade', 8, 3, 3)
# Logger.activate(Logger)

# Definir a função de callback
def simulacao_callback_bqc(weights, loss):
    """
    Esta função é chamada a cada passo do otimizador.
    Ela simula o envio do circuito com os pesos atuais para uma rede BQC.
    O 'passo' é gerenciado externamente pelo mecanismo de callback do classificador.
    """
    # Usamos uma variável global ou de classe para rastrear a época, se necessário,
    # já que a assinatura básica do callback é apenas (weights, loss).
    global contador_epoca
    print(f"\n[Época {contador_epoca+1}] Loss: {loss:.4f} - Simulando envio do circuito para a rede...")
    
    # Em uma implementação real, você construiria e enviaria o circuito:
    # current_circuit = feature_map.compose(ansatz.assign_parameters(weights))
    # try:
    #     rede.application_layer.run_app(...)
    #     print(f"[Época {contador_epoca+1}] Simulação de envio concluída.")
    # except Exception as e:
    #     print(f"[Época {contador_epoca+1}] Erro ao enviar circuito: {str(e)}")
    print(f"[Época {contador_epoca+1}] Simulação de envio concluída.")
    contador_epoca += 1


### **Parte 6: Treinamento do Classificador VQC**

Com todos os componentes prontos, iniciamos o treinamento do modelo.

1.  **Otimizador SPSA:** O `SPSA` (Simultaneous Perturbation Stochastic Approximation) é um otimizador clássico especialmente eficiente para ambientes ruidosos, como o de computadores quânticos. Ele ajusta os pesos do *ansatz* para minimizar a função de perda. O número de iterações (`maxiter`) foi limitado a 100, pois a simulação completa (incluindo o callback) é computacionalmente intensiva.

2.  **NeuralNetworkClassifier:** Esta é a classe do Qiskit que gerencia todo o processo de treinamento. Nós a instanciamos com:
    * `neural_network=qnn`: Nosso modelo quântico.
    * `optimizer=optimizer`: O otimizador SPSA.
    * `loss=CrossEntropyLoss()`: Uma função de perda padrão para problemas de classificação multiclasse.
    * `callback=simulacao_callback_bqc`: **O ponto-chave**. Passamos nossa função de callback aqui. Agora, a cada passo que o SPSA der para otimizar os pesos, a função será chamada, imprimindo o *loss* e simulando o envio do circuito.

3.  **Execução do Treinamento:** A linha `vqc.fit(X_train, y_train)` inicia o loop de treinamento. O classificador começa a alimentar o modelo com os dados de treino, calcular a perda e usar o otimizador para ajustar os pesos, chamando o callback a cada passo.

In [ ]:
# Passo 6: Treinamento com Callback de BQC
print("\nTREINANDO O VQC COM RUÍDO E CALLBACK DE BQC...")
optimizer = SPSA(maxiter=100) # Menos iterações pois a simulação pode ser lenta

# Inicializar contador de época para o callback
contador_epoca = 0

# Passar a função de callback para o classificador
vqc = NeuralNetworkClassifier(
    neural_network=qnn,
    optimizer=optimizer,
    loss=CrossEntropyLoss(),
    one_hot=True,
    callback=simulacao_callback_bqc # A função é passada aqui
)

start_time = time.time()
vqc.fit(X_train, y_train)
end_time = time.time()
print(f"\nTreinamento concluído em {end_time - start_time:.2f} segundos.")

### **Parte 7: Avaliação do Modelo e Métricas Finais**

Após o treinamento, é crucial avaliar o quão bem o nosso modelo VQC se generaliza para dados que ele nunca viu antes (o conjunto de teste).

1.  **Previsão:** O método `vqc.predict(X_test)` usa o modelo treinado para fazer previsões no conjunto de teste. O resultado (`y_pred_onehot`) está no formato one-hot.

2.  **Decodificação:** Para calcular as métricas, os rótulos previstos e os rótulos de teste verdadeiros são convertidos de volta do formato one-hot para o formato de rótulo único (0, 1 ou 2) usando `np.argmax`.

3.  **Cálculo de Métricas:** São calculadas quatro métricas de classificação padrão:
    * **Acurácia:** Percentual de previsões corretas.
    * **Precisão:** Dentre todas as vezes que o modelo previu uma classe, quantas estavam corretas.
    * **Recall:** Dentre todas as instâncias reais de uma classe, quantas o modelo conseguiu identificar.
    * **F1 Score:** Média harmônica entre precisão e recall, útil para dados desbalanceados.

4.  **Matriz de Confusão:** Por fim, a `confusion_matrix` é gerada e exibida. Ela é uma tabela que visualiza o desempenho do classificador, mostrando os acertos e os erros para cada classe. Por exemplo, ela mostra quantas vezes a classe "setosa" foi classificada corretamente e quantas vezes foi confundida com "versicolor" ou "virginica".

In [ ]:
# Passo 7: Avaliação do Modelo
print("\nCALCULANDO MÉTRICAS DE CLASSIFICAÇÃO...")
y_pred_onehot = vqc.predict(X_test)

# Decodificar previsões one-hot
y_pred_labels = np.argmax(y_pred_onehot, axis=1)
y_test_labels = np.argmax(y_test, axis=1)

# Calcular métricas
accuracy = accuracy_score(y_test_labels, y_pred_labels)
precision = precision_score(y_test_labels, y_pred_labels, average='weighted', zero_division=0)
recall = recall_score(y_test_labels, y_pred_labels, average='weighted', zero_division=0)
f1 = f1_score(y_test_labels, y_pred_labels, average='weighted', zero_division=0)

print("\nMÉTRICAS FINAIS:")
print(f"Acurácia : {accuracy:.4f}")
print(f"Precisão : {precision:.4f}")
print(f"Recall   : {recall:.4f}")
print(f"F1 Score : {f1:.4f}")

# Exibir matriz de confusão
print("\nMatriz de Confusão:")
cm = confusion_matrix(y_test_labels, y_pred_labels)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=iris.target_names)
disp.plot()
plt.show()